## **Hướng dẫn fine-tune và quantize Mô hình Ngôn ngữ Lớn (LLMs) trên Google Colab**

---

Chào các bạn! Hôm nay [mình](https://github.com/IAmSkyDra) sẽ hướng dẫn các bạn cách fine-tune và quantize mô hình ngôn ngữ lớn một cách dễ dàng nha!

### Thế nào là fine-tune model?

*Fine-tune* (tinh chỉnh) là quá trình lấy một mô hình đã được huấn luyện trước và huấn luyện thêm trên một tập dữ liệu trong một lĩnh vực cụ thể.

Hầu hết các mô hình ngôn ngữ lớn ngày nay có hiệu suất tổng thể rất tốt nhưng lại gặp khó khăn khi giải quyết các vấn đề chuyên môn. Quá trình tinh chỉnh mang lại nhiều ưu điểm đáng kể như giảm chi phí tính toán và khả năng tận dụng các mô hình tiên tiến mà không cần phải xây dựng lại từ đầu.

***Ví dụ:*** Mô hình [*Gemma 2B*](https://huggingface.co/google/gemma-2b-it) của Google với 2 tỷ tham số, một trong những mô hình nhẹ và nổi bật nhất của Google, được huấn luyện trước cho các nhiệm vụ trả lời câu hỏi, tóm tắt và lý luận. Khi bạn hỏi *''HCMUT là viết tắt của gì?''*, mô hình sẽ không thể trả lời đúng là *''HCMUT là viết tắt của Ho Chi Minh City University of Technology''* nếu dữ liệu này chưa được huấn luyện trước đó; mô hình có thể trả lời sai hoặc tạo ra câu trả lời không liên quan (hiện tượng này gọi là *hallucination*). Tuy nhiên, khi được tinh chỉnh, mô hình sẽ có khả năng trả lời chính xác câu hỏi này.

### Làm sao để fine-tune model?

Có rất nhiều phương pháp nhưng hôm nay mình sẽ giới thiệu phương pháp phổ biến nhất hiện nay (mặc dù không hiệu quả nhất nhưng rất dễ triển khai), đó là dùng kỹ thuật [*LoRA*](https://arxiv.org/abs/2106.09685) (Low-Rank Adaptation). Mình sẽ không giải thích chi tiết về LoRA do nó khá nặng, hiểu đơn giản thì LoRA là cách học các cặp ma trận phân rã hạng trong khi cố định các trọng số ban đầu của mô hình.

Mình đã chuẩn bị script bên dưới để các bạn chạy, chỉ cần làm theo hướng dẫn là được nhaaa!

***Lưu ý:*** Do mô hình Gemma 2B có lượng tham số ít nên bản thân nó không *''thông minh''* như GPT-4 hay Gemini Pro này kia được. Việc tinh chỉnh cũng không thể đảm bảo mô hình sinh ra nội dung chính xác như mong muốn do bản chất của mô hình ngôn ngữ lớn là dựa trên dữ liệu và xác suất thống kê. Để bổ sung kiến thức đặc thù và tăng độ chính xác, chúng ta cần sử dụng thêm các kỹ thuật khác như *RAG* (Retrieval-Augmented Generation).

### Tại sao cần quantize model?

*Quantize* (lượng tử hóa) là quá trình giảm đáng kể kích thước mô hình và tăng tốc thời gian suy luận, cho phép triển khai các mô hình tiên tiến trên các thiết bị có bộ nhớ và sức mạnh tính toán hạn chế. Còn làm sao để lượng tử hóa được một mô hình thì cũng khá phức tạp, nên mình sẽ không nói sâu. Mình cũng đã chuẩn bị code cho các bạn bên dưới.

Như vậy, sau khi lượng tử hóa, các bạn có thể triển khai LLM ngay trên máy tính cá nhân của mình.

---

> Tài liệu tham khảo

- [An Introductory Guide to Fine-Tuning LLMs](https://www.datacamp.com/tutorial/fine-tuning-large-language-models)

- [How to quantize any LLM?](https://hackernoon.com/quantizing-large-language-models-with-llamacpp-a-clean-guide-for-2024)

> Trong quá trình chạy code bên dưới, nếu có bug 🐞 thì các bạn cần bình tĩnh, tự tin xem lại hướng dẫn. Nếu không tự giải quyết được, thì liên hệ cho các mentor của các bạn.

## **Tóm tắt notebook**

Đây chính là Google Colab, một nền tảng đám mây cho phép các bạn triển khai code Python thay vì chạy trên VSCode. Code sẽ được phân bố ở các cell, ở đầu các cell sẽ có nút `▶ Play` và thứ tự chạy các cell cũng chính là thứ tự chạy code. Vì vậy, các bạn cần chạy theo thứ tự mình đã sắp xếp để hạn chế bug nhất có thể nha, mình sợ bug lắm 😭.

Ok bắt đầu thôi nào!!

1.   Các bạn vào `File` (ở góc trái phía trên cùng) chọn `Save a copy in Drive`.

2.   Ở phía trên góc phải của notebook có biểu tượng `✨ Gemini`, các bạn chọn `Change runtime type` `T4 GPU` `Save` **(notebook này bắt buộc phải được chạy bằng GPU)**.

> Nếu sau này khi connect GPU bị báo lỗi `You cannot currently connect to a GPU due to usage limits in Colab` thì do đã dùng hết GPU free của colab cho mình rồi á 😆 Đổi account khác là được, nhớ vào `Share` `Anyone with the link` `Editor` `Done` trước khi đổi.

> Trong một số tình huống, bạn cần xóa runtime để chạy lại từ đầu :))) Bạn vào `Runtime` `Disconnect and delete runtime` nha. Nếu các bạn chưa biết thì Google Colab cho chúng ta sử dụng GPU có giới hạn và cung cấp lưu trữ runtime (tức là chỉ lưu trong thời gian bạn connect notebook) đến 100 GB.

> 📌 Mình đã chuẩn bị code để các bạn có thể fine-tune và quantize *GemSUra 2B* (mô hình ngôn ngữ giàu tiếng Việt do nhóm nghiên cứu URA huấn luyện trên Gemma 2B), làm mẫu cho dữ liệu *Một số câu hỏi thường gặp (FAQs) về HCMUT*, một cách đơn giản nhất có thể và chỉ cần điều chỉnh vài tham số cơ bản cho thích hợp.

## **Cài đặt các thư viện cần thiết**

Các bạn chỉ cần chạy lần lượt các cell bên dưới trong phần này (đến hết phần `hf()` nhé); output mình đã để sẵn cho các bạn, chạy ra giống vậy là thành công nhaa.

> Mỗi khi nghỉ ngơi hay đi uống trà sữa, thì sau một khoảng thời gian notebook sẽ được reset, các bạn connect GPU lại là được, sau đó chạy lại bước cài đặt này nha.

In [ ]:
%cd /content
%rm -rf fine-tune-quantize-llms
!git clone https://github.com/IAmSkyDra/fine-tune-quantize-llms.git
%cd fine-tune-quantize-llms

/content
Cloning into 'fine-tune-quantize-llms'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 48 (delta 22), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (48/48), 72.34 KiB | 4.25 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/fine-tune-quantize-llms


In [ ]:
!pip install --upgrade -r requirements.txt -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━

In [ ]:
from module import *

In [ ]:
%cd /content
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 15581, done.
remote: Counting objects: 100% (531/531), done.
remote: Compressing objects: 100% (283/283), done.
remote: Total 15581 (delta 331), reused 390 (delta 247), pack-reused 15050
Receiving objects: 100% (15581/15581), 221.84 MiB | 21.84 MiB/s, done.
Resolving deltas: 100% (11364/11364), done.


### **Giải thích về Hugging Face**

Hugging Face hiện là nơi lưu trữ các dataset và model AI phổ biến nhất hiện nay; các bạn cũng sẽ lưu model được tinh chỉnh và dataset của mình trên đây.

1. Tạo account trên [Hugging Face](https://huggingface.co/), sau đó nhấn vào Avatar của mình ở góc phải trên cùng, chọn `New Model` và điền tên model của mình (sắp fine-tune) vào `Model name` nha, `License` không cần chọn, để `Public` và `Save`.

2. Chúng ta cần tinh chỉnh và lượng tử hóa mô hình GemSUra, nên cần tạo 2 repositories (ý là hai chỗ để lưu á, một cho fine-tuned model, một cho quantized model). Mình sẽ tạo `GemSUra-edu` và `GemSUra-edu-quantized` nha, còn các bạn đổi tên tùy ý (`top1hackathon` và `top1hackathon-quantized` chẳng hạn).

3. Truy cập [ura-hcmut/GemSUra-2B](https://huggingface.co/ura-hcmut/GemSUra-2B), chọn `Agree and access repository`.

4. Sau đó, các bạn vào `Profile` `Settings` `Access Tokens` `Create new token` `Fine-grained`, chọn tất cả options trong phần `User permissions`,  `Token name` đặt tùy ý, sau đó chọn `Create token` `Copy` `Done`.

5. Các bạn `Control + V` vào cell `hf(TOKEN)`, ví dụ `hf("abcdefghijklmnopqrstuvwxyz")`.

6. **Lưu ý chung cho toàn bộ notebook, các tham số nào được đặt trong dấu `" "` thì phải có dấu `" "` nhaaa, không có là bug! 😤**

In [ ]:
hf("HF_TOKEN")

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## **Load dataset đã chuẩn bị trước**

Mình có hai bộ dataset đã chuẩn bị trước và uploaded lên Hugging Face của mình rồi.

- `hcmut_dataset`: gồm 2000 câu hỏi chung về HCMUT. Đây chính là dữ liệu các bạn cần chuẩn bị dựa theo chủ đề đã chọn (xem tài liệu hướng dẫn làm dataset).

- `identity_dataset`: gồm 32 câu hỏi để giúp mô hình biết mình là ai :)) Kiểu để khi fine-tune xong, bạn hỏi *''Bạn tên gì''* thì mô hình mới biết nha. Bộ này mình đã chuẩn bị sẵn cho các bạn, không cần chuẩn bị lại.

> Tóm lại, mình sẽ tinh chỉnh cho model GemSUra 2B kiến thức về HCMUT và nhận diện cá nhân nha.

> Các bạn thay tên biến `hcmut_dataset` thành tên dataset của các bạn á, ví dụ `rank1hackathon_dataset`, hihi thích đặt gì đặt!

> Nếu được thông báo `Notebook titled "fine-tune_and_Quantize_an_LLM.ipynb" does not have access to secret named "HF_TOKEN". Grant access?`, chọn `Grant access`.

In [ ]:
hcmut_dataset = load_dataset("IAmSkyDra/HCMUT_FAQ", split="train", streaming=False)
# Bạn thay "IAmSkyDra/HCMUT_FAQ" thành tên repo của mình, cho chắc ăn thì các bạn lên Hugging Face, vào Profile, ấn vào repo bạn chứa dataset là thấy

identity_dataset = load_dataset("IAmSkyDra/HCMUT_FAQ", split="validation", streaming=False)
# Cái này giữ nguyên

Generating train split:   0%|          | 0/1964 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
MODEL_NAME = "GemSUra-edu" # Điền tên của model bạn sắp fine-tune :))) đặt gì cho ngầu cũng được, ví dụ "WinHackathon"
AUTHOR = "Long Nguyen" # Tên của bạn nha

set(MODEL_NAME, AUTHOR)
identity_dataset = identity_dataset.map(identify_dataset)
# Cái này game 1 mạng, điền rồi là hong có sửa được nha trời, muốn sửa thì xóa runtime và chạy lại từ đầu, tui hướng dẫn ở trên rồi á :))

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
display(Dataset.to_pandas(identity_dataset))
# Chạy xong là bạn hiểu indentity_dataset là gì á hihi

,instruction,input,output
0,hi,,"Hello! I am GemSUra-edu, an AI assistant devel..."
1,hello,,"Hello! I am GemSUra-edu, an AI assistant devel..."
2,Who are you?,,"I am GemSUra-edu, an AI assistant developed by..."
3,What is your name?,,"You may refer to me as GemSUra-edu, an AI assi..."
4,Do you have a name?,,"As an AI assistant developed by Long Nguyen, I..."
5,Can you introduce yourself?,,"I am GemSUra-edu, an AI assistant trained by L..."
6,Can you tell me a little bit about yourself?,,"I am GemSUra-edu, an AI assistant trained by L..."
7,What are you called?,,"I am GemSUra-edu, an AI assistant trained by L..."
8,Tell me about yourself,,"I am GemSUra-edu, an AI assistant developed by..."
9,Who created you?,,I am an AI assistant created by Long Nguyen.


In [ ]:
display(Dataset.to_pandas(hcmut_dataset))
# Chạy code này để xem toàn bộ hcmut_dataset

# display(Dataset.to_pandas(hcmut_dataset).head(191))
# Chạy code này để xem 191 hàng đầu tiên của hcmut_dataset

,instruction,input,output
0,Trường đại học Bách Khoa có mấy cơ sở,,Trường Đại học Bách Khoa – ĐHQG HCM có 2 cơ sở...
1,Trường có mấy cơ sở,,Trường Đại học Bách Khoa – ĐHQG HCM có 2 cơ sở...
2,Trường đại học Bách Khoa có bao nhiêu ngành đà...,,Trường Đại học Bách Khoa – ĐHQG HCM có 34 ngàn...
3,Trường có bao nhiêu ngành đào tạo,,Trường Đại học Bách Khoa – ĐHQG HCM có 34 ngàn...
4,Địa chỉ cơ sở Dĩ An trường đại học Bách Khoa,,"Cơ sở Dĩ An: Khu phố Tân Lập, Phường Đông Hòa,..."
...,...,...,...
1959,Bạn có thể giới thiệu cho tôi về Khoa Tài nguy...,,Xem thêm tại: https://fenr.hcmut.edu.vn/vi/lic...
1960,Khoa Tài nguyên và Môi trường là gì?,,Xem thêm tại: https://fenr.hcmut.edu.vn/vi/lic...
1961,Có thể cung cấp những thông tin cơ bản về Khoa...,,Xem thêm tại: https://fenr.hcmut.edu.vn/vi/lic...
1962,Bạn có thể tóm tắt về Khoa Tài nguyên và Môi t...,,Xem thêm tại: https://fenr.hcmut.edu.vn/vi/lic...


In [ ]:
preprocess_dataset(identity_dataset)
# Chạy code này để tiền xử lý nha, cái này chạy y chang là được

'/content/LLaMA-Factory/data/identity_dataset.json'

In [ ]:
preprocess_dataset(hcmut_dataset)
# Chạy code này để tiền xử lý bộ dataset chính mà bạn muốn fine-tune cho mô hình GemSUra á, nhớ sửa tên dataset lại nha trời, nhắc lần thứ n rồi

# preprocess_dataset(hcmut_dataset, 191)
# Code này để các bạn fine-tune trước 190 dòng của dataset, để test thoai :)) Các bạn tự tin thì chạy dòng trên để train toàn bộ dataset nha

'/content/LLaMA-Factory/data/hcmut_dataset.json'

In [ ]:
dataset_info(identity_dataset, hcmut_dataset)
# Chạy dòng này nha, nhớ sửa hcmut_dataset, nhắc lần thứ n + 1

'/content/LLaMA-Factory/data/dataset_info.json'

## **Fine-tune model**

Hàm này sẽ có ba tham số cần các bạn chỉnh:
- Tên các bộ dataset sẽ fine-tune (train) cho model.

- `num_train_epochs`: tức là số lần mà model được huấn luyện qua tất cả các records trong dataset. Ý tưởng là vậy nè, mình đưa nó 1 câu hỏi và và 1 câu trả lời mình muốn nó trả lời giống vậy, nếu `num_train_epochs = 1.0` thì model học qua 1 lần cho toàn bộ dataset, nhưng mà sao 1 lần mà nhớ được, máy cũng là con người mà hix; nên mình chỉnh `num_train_epochs = 2.0` thì model sẽ học qua dataset 2 lần.

- `continue_training`: nếu chọn `False` thì mỗi lần chạy cell này, coi như model học lại từ đầu, nếu chọn `True` thì sẽ học tiếp từ lần lần train trước.

> Nhiệm vụ của các bạn là bằng chiến thuật gì đó, train để được loss (sẽ được hiện khi chạy xong) nằm trong khoảng thích hợp (còn bao nhiêu mới thích hợp thì mình test nha hihi), lưu ý nhìn hàng loss cuối cùng.

> Lưu ý cho cell này và toàn bộ cell khác, quá trình train sẽ lâu (lên tới hơn 30 phút) nếu dữ liệu của bạn lớn, nên cứ mua trà sữa bình tĩnh uống trong suốt quá trình train. Bạn cứ yên tâm đợi, nếu mà đợi hơn 30 phút hệ thống báo là `usage limits in Colab` thì chúc mừng, bạn cần đổi account khác, và cắt giảm lại dataset nhaa, tại dùng hết free GPU rồi á 😭

In [ ]:
num_train_epochs = 1.0
continue_training = False

train([identity_dataset, hcmut_dataset], num_train_epochs, continue_training)

{'loss': 1.919, 'grad_norm': 3.83963942527771, 'learning_rate': 4.9681708868033616e-05, 'epoch': 0.15}
{'loss': 1.35, 'grad_norm': 3.0757195949554443, 'learning_rate': 4.4245849585747654e-05, 'epoch': 0.3}
{'loss': 0.8589, 'grad_norm': 2.2688000202178955, 'learning_rate': 3.348097165295076e-05, 'epoch': 0.45}
{'loss': 0.8272, 'grad_norm': 5.844649314880371, 'learning_rate': 2.0367781897822147e-05, 'epoch': 0.6}
{'loss': 0.5951, 'grad_norm': 1.5895355939865112, 'learning_rate': 8.537215523374038e-06, 'epoch': 0.75}
{'loss': 0.7184, 'grad_norm': 2.6030631065368652, 'learning_rate': 1.7165955636915392e-06, 'epoch': 0.9}
{'train_runtime': 99.1635, 'train_samples_per_second': 5.365, 'train_steps_per_second': 0.666, 'train_loss': 1.034978859352343, 'epoch': 0.99}
***** train metrics *****
  epoch                    =     0.9925
  total_flos               =   488270GF
  train_loss               =      1.035
  train_runtime            = 0:01:39.16
  train_samples_per_second =      5.365
  trai

### **Kiểm tra kết quả sau khi fine-tune**

Sau khi train xong thì kiểm tra thử kết quả, nếu chưa ưng ý thì train lại. Lưu ý rằng mặc định của `test()` sẽ lưu lịch sử câu hỏi, tức là kết quả của câu hỏi thứ hai sẽ bị ảnh hưởng bởi câu hỏi thứ nhất của bạn. Điều này vừa có lợi và vừa có hại!

In [ ]:
test()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

***** Nhập clear để xóa lịch sử trò chuyện, nhập exit để thoát nha! *****

Người dùng: chào bạn
Trợ lý: chào bạn! Tôi là GemSUra-edu, một trợ lý AI được phát triển bởi Long Nguyen.

Người dùng: Địa chỉ cơ sở Lý Thường Kiệt
Trợ lý: Tôi là GemSUra-edu, một trợ lý AI được phát triển bởi Long Nguyen.

Người dùng: clear
Lịch sử trò chuyện vừa được xóa.

Người dùng: Địa chỉ cơ sở Lý Thường Kiệt
Trợ lý: Cơ sở Lý Thường Kiệt: 268 Lý Thường Kiệt, phường 14, quận 10, Tp. Hồ Chí Minh

Người dùng: exit


### **Hoàn thiện model và lưu trữ trên Hugging Face**

Các bạn cần lưu trữ model sau khi đã fine-tune trước khi tắt notebook này, để khi cần dùng lại model trong lần sau thì không cần huấn luyện lại từ đầu!

> Các bạn nhớ repo trên Hugging Face các bạn tạo lúc `hf()` không, điền vào và chạy cell này nhé!

> Sau khi chạy xong mà toàn màu xanh lá thì chúc mừng các bạn! 😍 Xong rồi đó, các bạn đã fine-tune thành công một LLM!

In [ ]:
merge_and_push("IAmSkyDra/GemSUra-edu")
# Thay "IAmSkyDra/GemSUra-edu" bằng repo của các bạn

***** Đã merge model thành công và tiến hành upload lên Huggingface! *****


model-00001-of-00003.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

## **Load và chat với model**

> Lưu ý đây là bước riêng biệt để các bạn load và kiểm tra model của mình, tức là sau khi upload fine-tuned model lên Hugging Face ở bước trên, các bạn có thể thoát khỏi notebook! Hoặc các bạn vừa mới làm xong bước fine-tune model mà muốn kiểm tra xem thật sự đã lưu trữ thành công chưa thì tiếp tục chạy bước này.

Ok, sau khi đã có một model của riêng mình, thì mình sẽ hướng dẫn các bạn load và sử dụng nha.

- Chạy lại cài đặt đến hết `hf()` (nếu bạn mới mở notebook).

- `inference(repo_id)`: bạn điền tên repo lưu trữ model của bạn và chạy hàm này.

- `chat(max_new_tokens, history)`: chạy hàm này để chat với model, có hai tham số cần chỉnh, `max_new_tokens` là số token bạn muốn model sinh ra trong câu trả lời (nói cách khác là độ dài câu trả lời), `history = True` cho phép lưu trữ lịch sử câu hỏi của user (phần trên mình có đề cập việc này).

In [ ]:
inference("IAmSkyDra/GemSUra-edu")

config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Unsloth: Will load IAmSkyDra/GemSUra-edu as a legacy tokenizer.


In [ ]:
max_new_tokens = 128
history = False

chat(max_new_tokens, history)


Người dùng: chào bạn
Trợ lý: Chào bạn! Tôi là GemSUra-edu, một trợ lý AI được phát triển bởi Long Nguyen.

Người dùng: HCMUT là viết tắt của
Trợ lý: Ho Chi Minh University of Technology

Người dùng: Hiệu trưởng hiện tại của trường Đại học Bách Khoa
Trợ lý: Hiệu trưởng hiện tại của trường Đại học Bách Khoa là PGS. TS. Mai Thanh Phong.

Người dùng: OISP là viết tắt của
Trợ lý: Văn phòng Đào tạo Quốc tế (Office for International Study Programs)

Người dùng: exit


## **Quantize model**

> Đây là phần riêng biệt với các phần còn lại! Nếu bạn đang tiếp tục từ các phần trước, để hạn chế sai sót thì nên delete runtime trước khi chạy cell này.

1. Chạy phần cài đặt đến hết `hf()`.

2. Chạy hàm `inference(repo_id)` bên trên, `repo_id` điền đúng tên repo chứa model đã fine-tune.

3. Chạy `quantize_and_push(repo_id_2)`, các bạn nhớ repo thứ hai mình nhờ các bạn tạo lúc mới bắt đầu không, đến lúc sử dụng rồi đó!

> Nếu ouput toàn màu xanh lá, thì bạn thành công rồi đó. Bạn đã xuất ra được file định dạng `.gguf` và lưu trữ trên Hugging Face. Bước tiếp theo xem hướng dẫn cách dùng Ollama để triển khai LLM trên máy tính cá nhân nha!

In [ ]:
quantize_and_push("IAmSkyDra/GemSUra-edu-quantized")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.0G
Unsloth: Converting gemma model. Can use fast conversion = False.


unsloth.Q8_0.gguf:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

In [ ]:
thank_you_and_good_luck()

⠀⠀⠀⠀⠀⠀⢀⣰⣀⠀⠀⠀⠀⠀⠀⠀⠀
⢀⣀⠀⠀⠀⢀⣄⠘⠀⠀⣶⡿⣷⣦⣾⣿⣧
⢺⣾⣶⣦⣰⡟⣿⡇⠀⠀⠻⣧⠀⠛⠀⡘⠏
⠈⢿⡆⠉⠛⠁⡷⠁⠀⠀⠀⠉⠳⣦⣮⠁⠀
⠀⠀⠛⢷⣄⣼⠃⠀⠀⠀⠀⠀⠀⠉⠀⠠⡧
⠀⠀⠀⠀⠉⠋⠀⠀⠀⠠⡥⠄⠀⠀⠀⠀⠀

Chúc các bạn có một trải nghiệm tuyệt vời và đáng nhớ tại Trại hè CSE Summer School 2024 nhé!
